# Heuristic complete

In [ ]:
import time
import pandas as pd
from collections import defaultdict
import numpy as np

class ReforestationHeuristic:
    def __init__(self):
        # Constants
        self.PLANT_HA_EQUIVALENT = 0.0069
        self.MIN_PLANTS_WEEKDAY = 725  # Corresponds to 5ha
        self.MIN_PLANTS_SATURDAY = 0
        self.WORK_HOURS_WEEKDAY = 6
        self.WORK_HOURS_SATURDAY = 3
        self.PLANTING_COST_PER_PLANT = 20 # Cost of planting per plant
        self.NURSERY_TRUCK_COST = 4500
        self.DELIVERY_TRUCK_COST = 0
        self.NURSERY_TRUCK_CAPACITY = 8000
        self.ORDER_LEAD_TIME = 1 # Days for an order to arrive
        self.STORAGE_CAPACITY = 6400 # Max plants in storage at any time
        self.TREATMENT_CAPACITY = 1200  # Max plants that can be treated per day
        self.DELIVERY_TRUCK_CAPACITY = 80 # Max plants per delivery truck to a polygon
        self.PLANTS_PER_HOUR = 100 # Planting rate per hour per crew
        self.PLANTING_START_DAY = 3  # Plants can be planted from day 3 after arrival (inclusive)
        self.PLANTING_END_DAY = 7    # Plants must be planted by day 7 after arrival (inclusive)
        self.DAYS = 225 # Total simulation days

        # Penalization for trashing plants:
        # This is the *additional* cost incurred when a plant is trashed,
        # beyond its original purchase price (which is already accounted for when ordered).
        # Set high to strongly disincentivize trashing.
        self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST = 0 # Increased significantly

    def load_data(self):
        """Load and validate all input data from CSV files."""
        print("Loading data...")
        try:
            self.distance_matrix = pd.read_csv('Dataset/distance_times.csv', index_col=0)
            self.plantas_pol = pd.read_csv('Dataset/plantas_pol.csv', index_col=0)
            self.plantas_vivero = pd.read_csv('Dataset/Plantas_vivero.csv')
            self.plantas_vol = pd.read_csv('Dataset/Plantas_vol.csv')
            
            try:
                self.survival_rates = pd.read_csv('Dataset/Plantas_supervivencia.csv', index_col='Plant Type')
            except KeyError: # Fallback for different column name if needed
                self.survival_rates = pd.read_csv('Dataset/Plantas_supervivencia.csv', index_col='Plant_Type')

            # Convert column names to integers for easier lookup (e.g., 'Dia 3' or 'Día 3' -> 3)
            self.survival_rates.columns = [
                int(col.replace('Dia ', '').replace('Día ', '')) for col in self.survival_rates.columns
            ]
            
            # Validate data and define key entities
            self.polygons = self.distance_matrix.index.tolist()
            self.storage_polygon = 'P18' # Assuming 'P18' is the storage location
            self.nurseries = ['V1', 'V2', 'V3', 'V4']
            
            # Filter plant types to only include those present in all relevant data files
            self.plant_types = [pt for pt in self.plantas_pol.index 
                              if (pt in self.survival_rates.index and
                                  not self.plantas_vivero[self.plantas_vivero['Plant Type'] == pt].empty and
                                  not self.plantas_vol[self.plantas_vol['Plant Type'] == pt].empty)]
            
            print(f"Loaded data for {len(self.polygons)} polygons, {len(self.plant_types)} plant types.")
            
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            raise # Re-raise the exception to stop execution if data loading fails

    def initialize_structures(self):
        """Initialize data structures for tracking simulation state."""
        self.plant_requirements = defaultdict(dict)
        for p in self.plantas_pol.columns:
            for pt in self.plant_types:
                self.plant_requirements[p][pt] = self.plantas_pol.loc[pt, p]
        
        self.nursery_costs = {}
        for n in self.nurseries:
            self.nursery_costs[n] = {}
            for pt in self.plant_types:
                # Get nursery cost for each plant type. Use a very high cost if not available.
                cost = self.plantas_vivero[self.plantas_vivero['Plant Type'] == pt][n].values[0]
                self.nursery_costs[n][pt] = cost if not np.isnan(cost) else float('inf')
        
        # storage and in_transit now store original nursery cost per batch
        self.storage = defaultdict(list) # key=plant_type, value=list of (quantity, arrival_day, original_nursery_cost) tuples
        self.in_transit = defaultdict(list) # Plants ordered but not yet arrived (arrival_day, plant_type, qty, original_nursery_cost)
        
        self.orders = [] # List to store placed orders
        self.plantings = [] # List to store planting records
        self.total_cost = 0 # Running total of all costs
        self.total_planted = 0 # Running total of plants successfully planted
        self.planted_per_polygon = defaultdict(lambda: defaultdict(int))  # Track planted per polygon per species
        self.trashed_plants_count = 0 # Running total of trashed plants
        
        # New for average survival rate calculation
        self.total_expected_survivors = 0 # Sum of (quantity planted * survival_rate)
        self.total_quantity_planted_for_avg_survival = 0 # Sum of quantity planted for weighted average

    def get_day_type(self, day):
        """Return day type ('weekday', 'saturday', 'sunday') and available working hours."""
        day_of_week = day % 7
        if day_of_week == 5:  # Saturday
            return 'saturday', self.WORK_HOURS_SATURDAY
        elif day_of_week == 6:  # Sunday
            return 'sunday', 0
        else: # Monday to Friday
            return 'weekday', self.WORK_HOURS_WEEKDAY

    def _calculate_max_daily_planting_capacity(self, day):
        """
        Estimate the maximum number of plants that can be planted on a given day.
        This considers the bottleneck between treatment capacity and crew capacity.
        """
        day_type, work_hours = self.get_day_type(day)
        if work_hours == 0:
            return 0
        
        crew_capacity = self.PLANTS_PER_HOUR * work_hours # Plants per hour * total hours available
        return min(self.TREATMENT_CAPACITY, crew_capacity) # The actual bottleneck on any given day

    def order_plants(self, day):
        """
        Orders plants with a strong emphasis on minimizing trashing by
        more accurately simulating future capacity consumption and penalizing potential waste.
        Survival rate is ignored for ordering decisions in this version.
        """
        current_total_plants_in_storage = sum(sum(qty for qty, _, _ in batches) for batches in self.storage.values())
        available_storage_space = self.STORAGE_CAPACITY - current_total_plants_in_storage
        
        if available_storage_space <= 0:
            # print(f"Day {day}: No available storage space for new orders.")
            return # Cannot order if storage is full
        
        # 1. Calculate total remaining requirements per plant type across all polygons
        total_remaining_requirements = defaultdict(int)
        for p in self.polygons:
            for pt in self.plant_types:
                planted = self.planted_per_polygon[p].get(pt, 0)
                total_remaining_requirements[pt] += max(0, self.plant_requirements[p].get(pt, 0) - planted)
        
        # If all requirements are met, no need to order more plants
        if sum(total_remaining_requirements.values()) == 0:
            # print(f"Day {day}: All plant requirements met. No new orders needed.")
            return

        # 2. Define the look-ahead horizon for capacity planning
        # This covers today's potential actions and future planting windows for newly arriving orders.
        look_ahead_horizon_start = day + 1 # Start from tomorrow
        # End day is when a plant ordered today would maximally be plantable (arrival + end of window)
        look_ahead_horizon_end = day + self.ORDER_LEAD_TIME + (self.PLANTING_END_DAY - 1) 
        
        daily_capacity_forecast = {} # Stores max capacity for each future day
        for d in range(look_ahead_horizon_start, look_ahead_horizon_end + 1):
            daily_capacity_forecast[d] = self._calculate_max_daily_planting_capacity(d)
        
        # 3. Simulate consumption of *existing* and *in-transit* stock to determine
        #    remaining capacity for new orders and to identify actual deficits.
        
        # Create a combined list of all current storage and in-transit plants
        all_plantable_batches = [] # Format: (expiration_day, plantable_start_day, qty, plant_type, original_nursery_cost)

        # Add plants currently in storage
        for pt, batches in self.storage.items():
            for qty, arrival_day, nursery_cost in batches:
                plantable_start_day = arrival_day + (self.PLANTING_START_DAY - 1)
                plantable_end_day = arrival_day + (self.PLANTING_END_DAY - 1)
                if plantable_end_day >= day: # Only include if not yet expired
                    all_plantable_batches.append((plantable_end_day, plantable_start_day, qty, pt, nursery_cost))
        
        # Add plants currently in transit
        for pt in self.plant_types:
            for arrival_day_scheduled, plant_type_in_transit, qty, nursery_cost in self.in_transit[pt]:
                plantable_start_day = arrival_day_scheduled + (self.PLANTING_START_DAY - 1)
                plantable_end_day = arrival_day_scheduled + (self.PLANTING_END_DAY - 1)
                # Only include if it arrives and becomes plantable within the look-ahead horizon
                if plantable_end_day >= day and arrival_day_scheduled <= look_ahead_horizon_end: 
                     all_plantable_batches.append((plantable_end_day, plantable_start_day, qty, plant_type_in_transit, nursery_cost))
        
        # Sort batches for simulated consumption:
        # 1. By expiration day (earliest expiring first - highest priority to use)
        # 2. By earliest plantable day (to use older, available stock first within its window)
        # 3. By quantity (smaller batches first, though less critical than expiration)
        all_plantable_batches.sort(key=lambda x: (x[0], x[1], x[2]))

        # Simulate "greedy consumption" of existing/in-transit stock using projected daily capacity
        # This determines how much of the future capacity is "pre-booked" by current plants.
        projected_consumed_from_existing_and_transit = defaultdict(int)
        remaining_daily_capacity = daily_capacity_forecast.copy() # Make a copy to decrement

        for exp_day, plantable_start_day, qty, pt, nursery_cost in all_plantable_batches:
            if qty <= 0: continue

            current_batch_qty = qty
            
            # Try to plant this batch on days within its plantable window and the forecast horizon,
            # prioritizing earlier days to free up capacity later.
            for current_forecast_day in range(max(look_ahead_horizon_start, plantable_start_day), min(look_ahead_horizon_end, exp_day) + 1):
                
                if remaining_daily_capacity.get(current_forecast_day, 0) > 0:
                    
                    plant_this_day_qty = min(current_batch_qty, remaining_daily_capacity[current_forecast_day])
                    
                    projected_consumed_from_existing_and_transit[pt] += plant_this_day_qty
                    remaining_daily_capacity[current_forecast_day] -= plant_this_day_qty
                    current_batch_qty -= plant_this_day_qty
                    
                    if current_batch_qty <= 0: # This batch is fully accounted for
                        break

        # Calculate the *net available capacity for new orders* after existing stock is provisionally consumed.
        # This is the total "empty slots" in our future planting schedule.
        total_remaining_capacity_for_new_orders = sum(remaining_daily_capacity.values())
        
        # 4. Calculate the net quantity to order for each plant type, considering trashing penalty
        orders_to_consider = [] # List of potential orders to evaluate
        
        for pt in self.plant_types:
            # How many plants of this type are still needed to meet total polygon requirements?
            net_needed_for_polygon_completion = total_remaining_requirements[pt] 
            
            # How many plants of this type are we projected to handle from existing/in-transit stock?
            covered_by_existing_simulated = projected_consumed_from_existing_and_transit[pt]
            
            # The true deficit we need to fill with *new* orders for this specific plant type.
            net_deficit_for_new_order = max(0, net_needed_for_polygon_completion - covered_by_existing_simulated)

            if net_deficit_for_new_order <= 0: 
                continue # No need to order this plant type if deficit is zero or negative

            # Determine the maximum quantity we *could* order for this plant type,
            # limited by its deficit, nursery truck capacity, and available storage space.
            potential_order_qty_for_analysis = min(
                net_deficit_for_new_order, 
                self.NURSERY_TRUCK_CAPACITY, 
                available_storage_space
            )
            
            if potential_order_qty_for_analysis <= 0:
                continue # Cannot order this quantity

            # Find the cheapest nursery for this plant type
            best_nursery = None
            best_base_cost = float('inf')
            
            for n in self.nurseries:
                nursery_cost = self.nursery_costs[n].get(pt, float('inf'))
                if nursery_cost < best_base_cost:
                    best_nursery = n
                    best_base_cost = nursery_cost
            
            if best_nursery is None:
                continue # No nursery provides this plant type

            # --- MODIFIED: Survival rate is ignored for ordering decisions ---
            survival_rate_for_new_order = 1.0 # Assume 100% survival for decision-making
            # Original line (commented out):
            # survival_rate_for_new_order = self.survival_rates.loc[pt, self.PLANTING_START_DAY] \
            #     if self.PLANTING_START_DAY in self.survival_rates.columns and pt in self.survival_rates.index else 0.0
            # ---------------------------------------------------------------
            
            if survival_rate_for_new_order <= 0.0: # Still check, just in case (though it will always be 1.0 here)
                continue # Don't order if it won't survive
            
            # Estimate trashing risk for the *proposed new order*
            # If the quantity we are *about to order* exceeds the remaining overall capacity for new orders,
            # then that excess portion is deemed likely to be trashed.
            # This is a critical step to penalize over-ordering.
            expected_trashed_from_this_order = max(0, potential_order_qty_for_analysis - total_remaining_capacity_for_new_orders)

            # Calculate the 'effective' cost per plant. If any part of this order is expected to be trashed,
            # we apply the full additional trashing penalty to the per-plant cost for decision-making.
            effective_cost_per_plant = best_base_cost + (self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST if expected_trashed_from_this_order > 0 else 0)

            # Calculate cost per *surviving* planted unit. This metric helps prioritize orders.
            # A higher effective cost (due to trashing penalty) will make this less attractive.
            # --- MODIFIED: Division by 1.0 (survival_rate_for_new_order) changes nothing ---
            cost_per_surviving_planted_unit = effective_cost_per_plant / survival_rate_for_new_order
            # --------------------------------------------------------------------------------

            orders_to_consider.append({
                'plant_type': pt,
                'qty_to_order': potential_order_qty_for_analysis, # This is the max quantity to consider for this type
                'nursery': best_nursery,
                'base_cost_per_unit': best_base_cost,
                'effective_cost_per_plant': effective_cost_per_plant,
                'cost_per_surviving_planted_unit': cost_per_surviving_planted_unit,
                'survival_rate': survival_rate_for_new_order, # This will be 1.0 for decision-making
                'expected_trashed_from_this_order': expected_trashed_from_this_order 
            })
        
        # 5. Sort potential orders by the calculated 'cost_per_surviving_planted_unit' (ascending)
        # This prioritizes the most cost-effective and least risky orders first.
        orders_to_consider.sort(key=lambda x: x['cost_per_surviving_planted_unit'])

        # 6. Place orders, strictly respecting all remaining capacities
        total_ordered_this_day = 0

        for order_info in orders_to_consider:
            # Stop if no storage space or no overall capacity for new orders remains
            if available_storage_space <= 0 or total_remaining_capacity_for_new_orders <= 0:
                break
            
            pt = order_info['plant_type']
            nursery = order_info['nursery']
            base_cost = order_info['base_cost_per_unit']

            # The actual quantity to order is the minimum of:
            # 1. The calculated 'qty_to_order' for this specific plant type (deficit capped by truck/storage)
            # 2. The *remaining* overall capacity for new orders across all types (prevents global over-ordering)
            # 3. The nursery truck capacity (explicitly added for clarity, though part of qty_to_order)
            
            order_qty = min(
                order_info['qty_to_order'],
                total_remaining_capacity_for_new_orders,
                self.NURSERY_TRUCK_CAPACITY 
            )
            
            if order_qty <= 0:
                continue # Skip if no quantity can be ordered

            # Place the order
            self.orders.append({
                'day': day,
                'nursery': nursery,
                'plant_type': pt,
                'quantity': order_qty,
                'arrival_day': day + self.ORDER_LEAD_TIME
            })
            
            # Add to in-transit inventory, storing original nursery cost for trashing calculation
            self.in_transit[pt].append((day + self.ORDER_LEAD_TIME, pt, order_qty, base_cost))
            # Add purchase cost and truck cost to total cost
            self.total_cost += order_qty * base_cost + self.NURSERY_TRUCK_COST 
            
            # Update available capacities
            available_storage_space -= order_qty
            total_remaining_capacity_for_new_orders -= order_qty # Consume from the global capacity pool
            total_ordered_this_day += order_qty

            # print(f"Day {day}: Ordered {order_qty} {pt} from {nursery} for arrival Day {day + self.ORDER_LEAD_TIME} (Base Cost: ${best_base_cost:.2f}/plant). Remaining cap for new orders: {total_remaining_capacity_for_new_orders}")

    def process_arrivals(self, day):
        """
        Processes plant arrivals from in-transit to storage. 
        Handles storage capacity limits by delaying excess arrivals.
        """
        # First, remove expired plants to free up space for new arrivals
        self.remove_expired_plants(day)

        for pt in list(self.in_transit.keys()): # Iterate over a copy of keys as list might change
            new_in_transit_list = [] # To hold plants that didn't arrive or were partially received
            arrived_today = 0
            
            # Sort in-transit batches by scheduled arrival day to process earlier ones first
            self.in_transit[pt].sort(key=lambda x: x[0]) 

            for arrival_day_scheduled, plant_type, qty, nursery_cost in self.in_transit[pt]:
                if arrival_day_scheduled == day: # This batch is scheduled to arrive today
                    current_storage_qty = sum(sum(q for q, _, _ in batches) for batches in self.storage.values())
                    space_left = self.STORAGE_CAPACITY - current_storage_qty
                    
                    if space_left > 0:
                        qty_to_store = min(space_left, qty)
                        # Add to storage, retaining arrival day and original nursery cost
                        self.storage[plant_type].append((qty_to_store, day, nursery_cost)) 
                        arrived_today += qty_to_store
                        remaining_qty = qty - qty_to_store
                        if remaining_qty > 0:
                            # If not all could be stored, reschedule remaining for tomorrow
                            new_in_transit_list.append((day + 1, plant_type, remaining_qty, nursery_cost)) 
                            # print(f"Day {day}: Accepted {qty_to_store} of {qty} {plant_type} plants. {remaining_qty} will attempt to arrive tomorrow due to storage limits.")
                    else:
                        # If no space, reschedule entire batch for tomorrow
                        new_in_transit_list.append((day + 1, plant_type, qty, nursery_cost)) 
                        # print(f"Day {day}: Storage full, couldn't accept {qty} {plant_type} plants. Will attempt to arrive tomorrow.")
                else:
                    # If not scheduled for today, keep it in in-transit list
                    new_in_transit_list.append((arrival_day_scheduled, plant_type, qty, nursery_cost))
            
            self.in_transit[pt] = new_in_transit_list
            
            if arrived_today > 0:
                print(f"Day {day}: {arrived_today} plants arrived and stored.")
        
    def remove_expired_plants(self, day):
        """
        Removes plants from storage that have exceeded their 7-day planting window.
        Applies the trashing penalty to the total cost.
        """
        removed_count = 0
        for pt in list(self.storage.keys()): # Iterate over a copy of keys as list might change
            updated_batches = [] # To hold batches that are still valid
            for qty, arrival_day, original_nursery_cost in self.storage[pt]:
                last_planting_possible_day = arrival_day + (self.PLANTING_END_DAY - 1)
                
                if day > last_planting_possible_day: # If current day is beyond the last possible planting day
                    removed_count += qty
                    # Add the additional penalty cost for trashing
                    self.total_cost += qty * self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST 
                    self.trashed_plants_count += qty 
                    print(f"Day {day}: Trashed {qty} {pt} plants that arrived on day {arrival_day} (expired). Original cost: ${original_nursery_cost:.2f}/plant + additional trashing penalty: ${self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST:.2f}/plant.")
                else:
                    updated_batches.append((qty, arrival_day, original_nursery_cost))
            self.storage[pt] = updated_batches
            if not updated_batches:
                del self.storage[pt] # Remove plant type from storage if no batches remain

        if removed_count > 0:
            print(f"Day {day}: Total {removed_count} plants removed due to expiration.")

    def get_survival_rate(self, plant_type, current_day, arrival_day):
        """
        Get the survival rate for a plant_type based on the day of planting
        relative to its arrival day.
        
        MODIFIED: Returns 1.0 (100%) if within planting window, otherwise 0.0.
        The actual survival_rates data is ignored for decision-making but will be used for final reporting.
        """
        days_since_arrival = current_day - arrival_day + 1 # Calculate 1-indexed days since arrival
        
        # Check if planting is within the valid window
        if not (self.PLANTING_START_DAY <= days_since_arrival <= self.PLANTING_END_DAY):
            return 0.0 # Plant cannot be planted, effectively 0% survival
        
        # --- MODIFIED: Always return 1.0 if within planting window ---
        # The original line below would return the actual survival rate from the data:
        # planting_day_key = max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, days_since_arrival))
        # if plant_type in self.survival_rates.index and planting_day_key in self.survival_rates.columns:
        #    return self.survival_rates.loc[plant_type, planting_day_key]
        return 1.0 # Assume 100% survival for decision-making purposes when plantable
        # -------------------------------------------------------------


    def treat_and_plant_plants(self, day):
        """
        Manages the treatment and planting operations for a given day.
        Prioritizes plants close to expiration. Survival rate is ignored for prioritization.
        """
        day_type, max_hours = self.get_day_type(day)
        if day_type == 'sunday':
            # print(f"Day {day}: No planting on Sundays.")
            return # No operations on Sundays
            
        max_planting_minutes = max_hours * 60
        planting_minutes_used = 0
        
        eligible_plant_pool = [] # List of (priority_score, plant_type, qty, arrival_day_of_batch, original_nursery_cost)
        
        # Populate the pool with plants currently in storage that are eligible for planting
        for pt, batches in self.storage.items():
            for qty, arrival_day, original_nursery_cost in batches: 
                days_since_arrival = day - arrival_day + 1
                
                # Check if the plant is within its valid planting window
                if self.PLANTING_START_DAY <= days_since_arrival <= self.PLANTING_END_DAY:
                    # Retrieve the *actual* survival rate for reporting later, but not for decision making here
                    survival_day_key = max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, days_since_arrival))
                    if pt in self.survival_rates.index and survival_day_key in self.survival_rates.columns:
                        actual_current_day_survival_rate = self.survival_rates.loc[pt, survival_day_key]
                    else:
                        actual_current_day_survival_rate = 0.0

                    if actual_current_day_survival_rate > 0: # Still only consider if it has a chance to survive in reality
                        days_left_to_plant = (arrival_day + self.PLANTING_END_DAY - 1) - day
                        
                        # Priority score: lower 'days_left_to_plant' (more urgent)
                        # --- MODIFIED: Removed survival rate from priority score ---
                        priority_score = (days_left_to_plant, ) 
                        eligible_plant_pool.append((priority_score, pt, qty, arrival_day, original_nursery_cost, actual_current_day_survival_rate))
        
        # Sorts the pool based on the priority_score (ascending for days_left_to_plant)
        # Note: If two plants have the same days_left_to_plant, their relative order is stable (FIFO-ish based on input order)
        eligible_plant_pool.sort() 
        
        total_plants_in_eligible_pool = sum(item[2] for item in eligible_plant_pool) # Total quantity of eligible plants
        total_to_treat_today = min(
            total_plants_in_eligible_pool,
            self.TREATMENT_CAPACITY # Limited by daily treatment capacity
        )
        
        if total_to_treat_today <= 0:
            # print(f"Day {day}: No eligible plants available in storage to treat.")
            return # No plants to treat/plant today
        
        treated_today = 0 # Counter for plants treated on the current day
        
        # Iterate through the prioritized eligible plants
        for priority_score, pt, qty_in_batch, arrival_day_of_batch, original_nursery_cost, actual_survival_rate_for_record in eligible_plant_pool:
            # Stop if daily treatment capacity or working hours are exhausted
            if treated_today >= total_to_treat_today or planting_minutes_used >= max_planting_minutes:
                break 
            
            polygons_with_need = [] # List of (polygon, remaining_need)
            # Find polygons that need this plant type and where it can be planted
            for p in self.polygons:
                remaining_need = max(0, self.plant_requirements[p].get(pt, 0) - self.planted_per_polygon[p].get(pt, 0))
                if remaining_need > 0:
                    # We still check if the *actual* survival rate for this polygon/plant type is > 0 for this day.
                    # This prevents trying to plant where it's truly impossible to survive.
                    polygon_actual_survival_rate = self.survival_rates.loc[pt, max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, day - arrival_day_of_batch + 1))] \
                        if pt in self.survival_rates.index and max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, day - arrival_day_of_batch + 1)) in self.survival_rates.columns else 0.0

                    if polygon_actual_survival_rate > 0: 
                        polygons_with_need.append((p, remaining_need, polygon_actual_survival_rate)) # Pass actual survival for recording
            
            # Sort polygons by quantity needed (more first), as survival rate is not a factor for decision-making
            # We explicitly *do not* sort by the actual_survival_rate here for decision purposes.
            polygons_with_need.sort(key=lambda x: x[1], reverse=True) 

            plants_from_this_batch_planted = 0 # Counter for plants planted from the current batch
            
            for p, remaining_need_in_polygon, polygon_actual_survival_rate_for_record in polygons_with_need:
                # Stop if current batch is fully planted, or daily capacity/hours exhausted
                if plants_from_this_batch_planted >= qty_in_batch or treated_today >= total_to_treat_today or planting_minutes_used >= max_planting_minutes:
                    break
                
                # Calculate delivery time to the polygon
                delivery_time = 60 + 2 * self.distance_matrix.loc[self.storage_polygon, p]
                # Check if there's enough time left for this delivery
                if planting_minutes_used + delivery_time > max_planting_minutes:
                    continue # Skip if not enough time
                
                # Determine the quantity to plant in this polygon from this batch
                plant_qty = min(
                    remaining_need_in_polygon,        # Cannot plant more than polygon needs
                    qty_in_batch - plants_from_this_batch_planted, # Cannot plant more than available in this batch
                    total_to_treat_today - treated_today,          # Cannot exceed overall daily treatment capacity
                    self.DELIVERY_TRUCK_CAPACITY         # Cannot exceed delivery truck capacity
                )
                
                if plant_qty <= 0:
                    continue # Skip if no quantity to plant

                # Update storage: find the exact batch and reduce its quantity
                batch_found = False
                for i, (stored_qty, stored_arrival_day, stored_nursery_cost) in enumerate(self.storage[pt]):
                    # Match by arrival_day and original_nursery_cost to ensure it's the correct batch
                    if stored_arrival_day == arrival_day_of_batch and stored_nursery_cost == original_nursery_cost: 
                        self.storage[pt][i] = (stored_qty - plant_qty, stored_arrival_day, stored_nursery_cost)
                        if self.storage[pt][i][0] <= 0: # If batch quantity drops to zero or less, remove it
                            self.storage[pt].pop(i)
                        batch_found = True
                        break
                if not batch_found:
                    print(f"Warning: Batch not found for {pt} with arrival_day {arrival_day_of_batch} and cost {original_nursery_cost}. This should not happen if logic is sound.")

                plants_from_this_batch_planted += plant_qty
                treated_today += plant_qty
                
                # Record the planting event, using the actual survival rate for the record
                self.plantings.append({
                    'day': day,
                    'polygon': p,
                    'plant_type': pt,
                    'quantity': plant_qty,
                    'arrival_day_of_batch': arrival_day_of_batch,
                    'survival_rate_applied': polygon_actual_survival_rate_for_record 
                })
                
                self.total_planted += plant_qty
                self.planted_per_polygon[p][pt] += plant_qty
                # Add planting cost and delivery truck cost to total cost
                self.total_cost += plant_qty * self.PLANTING_COST_PER_PLANT + self.DELIVERY_TRUCK_COST 
                
                planting_minutes_used += delivery_time

                # Update for average survival rate calculation using the ACTUAL survival rate
                self.total_expected_survivors += plant_qty * polygon_actual_survival_rate_for_record
                self.total_quantity_planted_for_avg_survival += plant_qty
                
                # print(f"Day {day}: Planted {plant_qty} {pt} at {p} (from batch arr. day {arrival_day_of_batch}, survival: {polygon_actual_survival_rate_for_record:.2f}, delivery: {delivery_time} mins)")
        
        print(f"Day {day}: Total treated and planted today: {treated_today}/{total_to_treat_today}.")


    def run_heuristic(self):
        """Runs the complete heuristic simulation for the specified number of days."""
        print("Running heuristic solution...")
        start_time = time.time()
        
        self.load_data()
        self.initialize_structures()
        
        for day in range(self.DAYS):
            print(f"\n=== Day {day} ===")
            
            self.process_arrivals(day) # Process plants arriving today
            self.order_plants(day) # Place new orders
            self.treat_and_plant_plants(day) # Treat and plant plants from storage
            
            # Print daily summary
            current_storage_qty = sum(sum(qty for qty, _, _ in batches) for batches in self.storage.values())
            print(f"Current Storage: {current_storage_qty}/{self.STORAGE_CAPACITY}")
            print(f"Total planted so far: {self.total_planted}")
            print(f"Total trashed plants: {self.trashed_plants_count}")
            
            # Check if all requirements are met to potentially stop early
            all_met = True
            for p in self.polygons:
                for pt in self.plant_types:
                    req = self.plant_requirements[p].get(pt, 0)
                    planted = self.planted_per_polygon[p].get(pt, 0)
                    if planted < req:
                        all_met = False
                        break
                if not all_met:
                    break
            
            if all_met and current_storage_qty == 0 and sum(sum(qty for _, _, qty, _ in transit_items) for transit_items in self.in_transit.values()) == 0:
                print("\nAll plants have been planted and no pending deliveries! Stopping simulation early.")
                break
            elif all_met:
                print("\nAll required plants have been planted, but there are still plants in storage or transit. Continuing to clear existing stock.")
                # The order_plants function will naturally stop ordering if requirements are met,
                # so the simulation will continue to clear existing inventory.

        # Calculate average survival rate at the end of the simulation
        average_survival_rate = 0.0
        if self.total_quantity_planted_for_avg_survival > 0:
            average_survival_rate = self.total_expected_survivors / self.total_quantity_planted_for_avg_survival

        total_required = sum(sum(p.values()) for p in self.plant_requirements.values())
        completion = (self.total_planted / total_required) * 100 if total_required > 0 else 0
        
        print("\n=== Heuristic Solution Complete ===")
        print(f"Total Cost: ${self.total_cost:,.2f}")
        print(f"Plants Planted: {self.total_planted:,} of {total_required:,}")
        print(f"Completion: {completion:.1f}%")
        print(f"Total Trashed Plants: {self.trashed_plants_count:,}") 
        print(f"Average Survival Rate of Planted Plants: {average_survival_rate:.2%}") # Formatted as percentage
        print(f"Runtime: {time.time() - start_time:.2f} seconds")
        
        self.save_results(average_survival_rate) # Pass average survival rate to save_results

    def save_results(self, average_survival_rate):
        """Save simulation results to CSV files."""
        print("\nSaving results...")
        
        orders_df = pd.DataFrame(self.orders)
        if not orders_df.empty:
            orders_df.to_csv('heuristic_orders.csv', index=False)
            print(f"Saved {len(orders_df)} orders to heuristic_orders.csv.")
        else:
            print("No orders were placed, 'heuristic_orders.csv' not created.")
        
        planting_df = pd.DataFrame(self.plantings)
        if not planting_df.empty:
            planting_df.to_csv('heuristic_planting.csv', index=False)
            print(f"Saved {len(planting_df)} planting records to heuristic_planting.csv.")
        else:
            print("No plants were planted, 'heuristic_planting.csv' not created.")
        
        total_required = sum(sum(p.values()) for p in self.plant_requirements.values())
        summary = {
            'total_cost': self.total_cost,
            'total_planted': self.total_planted,
            'total_required': total_required,
            'completion_percentage': (self.total_planted / total_required) * 100 if total_required > 0 else 0,
            'total_trashed_plants': self.trashed_plants_count,
            'average_survival_rate_planted': average_survival_rate # Add to summary
        }
        pd.DataFrame([summary]).to_csv('heuristic_summary.csv', index=False)
        print("Saved summary to heuristic_summary.csv.")

if __name__ == "__main__":
    solver = ReforestationHeuristic()
    solver.run_heuristic()

# Heuristic small cases

In [ ]:
import time
import pandas as pd
from collections import defaultdict
import numpy as np
import random # Import the random module

class ReforestationHeuristic:
    def __init__(self):
        # Constants
        self.PLANT_HA_EQUIVALENT = 0.0069
        self.MIN_PLANTS_WEEKDAY = 725  # Corresponds to 5ha
        self.MIN_PLANTS_SATURDAY = 0
        self.WORK_HOURS_WEEKDAY = 6
        self.WORK_HOURS_SATURDAY = 3
        self.PLANTING_COST_PER_PLANT = 20 # Cost of planting per plant
        self.NURSERY_TRUCK_COST = 4500
        self.DELIVERY_TRUCK_COST = 0
        self.NURSERY_TRUCK_CAPACITY = 8000
        self.ORDER_LEAD_TIME = 1 # Days for an order to arrive
        self.STORAGE_CAPACITY = 6400 # Max plants in storage at any time
        self.TREATMENT_CAPACITY = 1200  # Max plants that can be treated per day
        self.DELIVERY_TRUCK_CAPACITY = 80 # Max plants per delivery truck to a polygon
        self.PLANTS_PER_HOUR = 100 # Planting rate per hour per crew
        self.PLANTING_START_DAY = 3  # Plants can be planted from day 3 after arrival (inclusive)
        self.PLANTING_END_DAY = 7    # Plants must be planted by day 7 after arrival (inclusive)
        self.DAYS = 225 # Total simulation days

        # Penalization for trashing plants:
        # This is the *additional* cost incurred when a plant is trashed,
        # beyond its original purchase price (which is already accounted for when ordered).
        # Set high to strongly disincentivize trashing.
        self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST = 0 # Increased significantly

    def load_data(self):
        """Load and validate all input data from CSV files."""
        print("Loading data...")
        try:
            self.distance_matrix = pd.read_csv('Dataset/distance_times.csv', index_col=0)
            self.plantas_pol = pd.read_csv('Dataset/plantas_pol.csv', index_col=0)
            self.plantas_vivero = pd.read_csv('Dataset/Plantas_vivero.csv')
            self.plantas_vol = pd.read_csv('Dataset/Plantas_vol.csv')
            
            try:
                self.survival_rates = pd.read_csv('Dataset/Plantas_supervivencia.csv', index_col='Plant Type')
            except KeyError: # Fallback for different column name if needed
                self.survival_rates = pd.read_csv('Dataset/Plantas_supervivencia.csv', index_col='Plant_Type')

            # Convert column names to integers for easier lookup (e.g., 'Dia 3' or 'Día 3' -> 3)
            self.survival_rates.columns = [
                int(col.replace('Dia ', '').replace('Día ', '')) for col in self.survival_rates.columns
            ]
            
            # Validate data and define key entities
            self.storage_polygon = 'P18' # Assuming 'P18' is the storage location
            self.nurseries = ['V1', 'V2', 'V3', 'V4']

            # --- MODIFICATION START ---
            all_available_polygons = self.distance_matrix.index.tolist()
            
            # Ensure 'P18' is in the list of all available polygons
            if self.storage_polygon not in all_available_polygons:
                raise ValueError(f"Storage polygon '{self.storage_polygon}' not found in distance matrix index.")

            # Remove 'P18' from the pool for random selection to avoid duplication
            selectable_polygons = [p for p in all_available_polygons if p != self.storage_polygon]
            
            # Determine how many more polygons are needed (15 total - 1 for P18)
            num_additional_polygons = 15 - 1 

            # Select 14 random polygons (or fewer if not enough available)
            # Use min to ensure we don't try to select more than available
            selected_random_polygons = random.sample(selectable_polygons, min(num_additional_polygons, len(selectable_polygons)))
            
            # Combine 'P18' with the randomly selected polygons
            self.polygons = [self.storage_polygon] + selected_random_polygons
            # --- MODIFICATION END ---
            
            # Filter plant types to only include those present in all relevant data files
            self.plant_types = [pt for pt in self.plantas_pol.index 
                              if (pt in self.survival_rates.index and
                                  not self.plantas_vivero[self.plantas_vivero['Plant Type'] == pt].empty and
                                  not self.plantas_vol[self.plantas_vol['Plant Type'] == pt].empty)]
            
            print(f"Loaded data for {len(self.polygons)} polygons, {len(self.plant_types)} plant types.")
            
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            raise # Re-raise the exception to stop execution if data loading fails

    def initialize_structures(self):
        """Initialize data structures for tracking simulation state."""
        self.plant_requirements = defaultdict(dict)
        for p in self.polygons:
            for pt in self.plant_types:
                self.plant_requirements[p][pt] = self.plantas_pol.loc[pt, p]
        
        self.nursery_costs = {}
        for n in self.nurseries:
            self.nursery_costs[n] = {}
            for pt in self.plant_types:
                # Get nursery cost for each plant type. Use a very high cost if not available.
                cost = self.plantas_vivero[self.plantas_vivero['Plant Type'] == pt][n].values[0]
                self.nursery_costs[n][pt] = cost if not np.isnan(cost) else float('inf')
        
        # storage and in_transit now store original nursery cost per batch
        self.storage = defaultdict(list) # key=plant_type, value=list of (quantity, arrival_day, original_nursery_cost) tuples
        self.in_transit = defaultdict(list) # Plants ordered but not yet arrived (arrival_day, plant_type, qty, original_nursery_cost)
        
        self.orders = [] # List to store placed orders
        self.plantings = [] # List to store planting records
        self.total_cost = 0 # Running total of all costs
        self.total_planted = 0 # Running total of plants successfully planted
        self.planted_per_polygon = defaultdict(lambda: defaultdict(int))  # Track planted per polygon per species
        self.trashed_plants_count = 0 # Running total of trashed plants
        
        # New for average survival rate calculation
        self.total_expected_survivors = 0 # Sum of (quantity planted * survival_rate)
        self.total_quantity_planted_for_avg_survival = 0 # Sum of quantity planted for weighted average

    def get_day_type(self, day):
        """Return day type ('weekday', 'saturday', 'sunday') and available working hours."""
        day_of_week = day % 7
        if day_of_week == 5:  # Saturday
            return 'saturday', self.WORK_HOURS_SATURDAY
        elif day_of_week == 6:  # Sunday
            return 'sunday', 0
        else: # Monday to Friday
            return 'weekday', self.WORK_HOURS_WEEKDAY

    def _calculate_max_daily_planting_capacity(self, day):
        """
        Estimate the maximum number of plants that can be planted on a given day.
        This considers the bottleneck between treatment capacity and crew capacity.
        """
        day_type, work_hours = self.get_day_type(day)
        if work_hours == 0:
            return 0
        
        crew_capacity = self.PLANTS_PER_HOUR * work_hours # Plants per hour * total hours available
        return min(self.TREATMENT_CAPACITY, crew_capacity) # The actual bottleneck on any given day

    def order_plants(self, day):
        """
        Orders plants with a strong emphasis on minimizing trashing by
        more accurately simulating future capacity consumption and penalizing potential waste.
        Survival rate is ignored for ordering decisions in this version.
        """
        current_total_plants_in_storage = sum(sum(qty for qty, _, _ in batches) for batches in self.storage.values())
        available_storage_space = self.STORAGE_CAPACITY - current_total_plants_in_storage
        
        if available_storage_space <= 0:
            # print(f"Day {day}: No available storage space for new orders.")
            return # Cannot order if storage is full
        
        # 1. Calculate total remaining requirements per plant type across all polygons
        total_remaining_requirements = defaultdict(int)
        for p in self.polygons:
            for pt in self.plant_types:
                planted = self.planted_per_polygon[p].get(pt, 0)
                total_remaining_requirements[pt] += max(0, self.plant_requirements[p].get(pt, 0) - planted)
        
        # If all requirements are met, no need to order more plants
        if sum(total_remaining_requirements.values()) == 0:
            # print(f"Day {day}: All plant requirements met. No new orders needed.")
            return

        # 2. Define the look-ahead horizon for capacity planning
        # This covers today's potential actions and future planting windows for newly arriving orders.
        look_ahead_horizon_start = day + 1 # Start from tomorrow
        # End day is when a plant ordered today would maximally be plantable (arrival + end of window)
        look_ahead_horizon_end = day + self.ORDER_LEAD_TIME + (self.PLANTING_END_DAY - 1) 
        
        daily_capacity_forecast = {} # Stores max capacity for each future day
        for d in range(look_ahead_horizon_start, look_ahead_horizon_end + 1):
            daily_capacity_forecast[d] = self._calculate_max_daily_planting_capacity(d)
        
        # 3. Simulate consumption of *existing* and *in-transit* stock to determine
        #    remaining capacity for new orders and to identify actual deficits.
        
        # Create a combined list of all current storage and in-transit plants
        all_plantable_batches = [] # Format: (expiration_day, plantable_start_day, qty, plant_type, original_nursery_cost)

        # Add plants currently in storage
        for pt, batches in self.storage.items():
            for qty, arrival_day, nursery_cost in batches:
                plantable_start_day = arrival_day + (self.PLANTING_START_DAY - 1)
                plantable_end_day = arrival_day + (self.PLANTING_END_DAY - 1)
                if plantable_end_day >= day: # Only include if not yet expired
                    all_plantable_batches.append((plantable_end_day, plantable_start_day, qty, pt, nursery_cost))
        
        # Add plants currently in transit
        for pt in self.plant_types:
            for arrival_day_scheduled, plant_type_in_transit, qty, nursery_cost in self.in_transit[pt]:
                plantable_start_day = arrival_day_scheduled + (self.PLANTING_START_DAY - 1)
                plantable_end_day = arrival_day_scheduled + (self.PLANTING_END_DAY - 1)
                # Only include if it arrives and becomes plantable within the look-ahead horizon
                if plantable_end_day >= day and arrival_day_scheduled <= look_ahead_horizon_end: 
                     all_plantable_batches.append((plantable_end_day, plantable_start_day, qty, plant_type_in_transit, nursery_cost))
        
        # Sort batches for simulated consumption:
        # 1. By expiration day (earliest expiring first - highest priority to use)
        # 2. By earliest plantable day (to use older, available stock first within its window)
        # 3. By quantity (smaller batches first, though less critical than expiration)
        all_plantable_batches.sort(key=lambda x: (x[0], x[1], x[2]))

        # Simulate "greedy consumption" of existing/in-transit stock using projected daily capacity
        # This determines how much of the future capacity is "pre-booked" by current plants.
        projected_consumed_from_existing_and_transit = defaultdict(int)
        remaining_daily_capacity = daily_capacity_forecast.copy() # Make a copy to decrement

        for exp_day, plantable_start_day, qty, pt, nursery_cost in all_plantable_batches:
            if qty <= 0: continue

            current_batch_qty = qty
            
            # Try to plant this batch on days within its plantable window and the forecast horizon,
            # prioritizing earlier days to free up capacity later.
            for current_forecast_day in range(max(look_ahead_horizon_start, plantable_start_day), min(look_ahead_horizon_end, exp_day) + 1):
                
                if remaining_daily_capacity.get(current_forecast_day, 0) > 0:
                    
                    plant_this_day_qty = min(current_batch_qty, remaining_daily_capacity[current_forecast_day])
                    
                    projected_consumed_from_existing_and_transit[pt] += plant_this_day_qty
                    remaining_daily_capacity[current_forecast_day] -= plant_this_day_qty
                    current_batch_qty -= plant_this_day_qty
                    
                    if current_batch_qty <= 0: # This batch is fully accounted for
                        break

        # Calculate the *net available capacity for new orders* after existing stock is provisionally consumed.
        # This is the total "empty slots" in our future planting schedule.
        total_remaining_capacity_for_new_orders = sum(remaining_daily_capacity.values())
        
        # 4. Calculate the net quantity to order for each plant type, considering trashing penalty
        orders_to_consider = [] # List of potential orders to evaluate
        
        for pt in self.plant_types:
            # How many plants of this type are still needed to meet total polygon requirements?
            net_needed_for_polygon_completion = total_remaining_requirements[pt] 
            
            # How many plants of this type are we projected to handle from existing/in-transit stock?
            covered_by_existing_simulated = projected_consumed_from_existing_and_transit[pt]
            
            # The true deficit we need to fill with *new* orders for this specific plant type.
            net_deficit_for_new_order = max(0, net_needed_for_polygon_completion - covered_by_existing_simulated)

            if net_deficit_for_new_order <= 0: 
                continue # No need to order this plant type if deficit is zero or negative

            # Determine the maximum quantity we *could* order for this plant type,
            # limited by its deficit, nursery truck capacity, and available storage space.
            potential_order_qty_for_analysis = min(
                net_deficit_for_new_order, 
                self.NURSERY_TRUCK_CAPACITY, 
                available_storage_space
            )
            
            if potential_order_qty_for_analysis <= 0:
                continue # Cannot order this quantity

            # Find the cheapest nursery for this plant type
            best_nursery = None
            best_base_cost = float('inf')
            
            for n in self.nurseries:
                nursery_cost = self.nursery_costs[n].get(pt, float('inf'))
                if nursery_cost < best_base_cost:
                    best_nursery = n
                    best_base_cost = nursery_cost
            
            if best_nursery is None:
                continue # No nursery provides this plant type

            # --- MODIFIED: Survival rate is ignored for ordering decisions ---
            survival_rate_for_new_order = 1.0 # Assume 100% survival for decision-making
            # Original line (commented out):
            # survival_rate_for_new_order = self.survival_rates.loc[pt, self.PLANTING_START_DAY] \
            #     if self.PLANTING_START_DAY in self.survival_rates.columns and pt in self.survival_rates.index else 0.0
            # ---------------------------------------------------------------
            
            if survival_rate_for_new_order <= 0.0: # Still check, just in case (though it will always be 1.0 here)
                continue # Don't order if it won't survive
            
            # Estimate trashing risk for the *proposed new order*
            # If the quantity we are *about to order* exceeds the remaining overall capacity for new orders,
            # then that excess portion is deemed likely to be trashed.
            # This is a critical step to penalize over-ordering.
            expected_trashed_from_this_order = max(0, potential_order_qty_for_analysis - total_remaining_capacity_for_new_orders)

            # Calculate the 'effective' cost per plant. If any part of this order is expected to be trashed,
            # we apply the full additional trashing penalty to the per-plant cost for decision-making.
            effective_cost_per_plant = best_base_cost + (self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST if expected_trashed_from_this_order > 0 else 0)

            # Calculate cost per *surviving* planted unit. This metric helps prioritize orders.
            # A higher effective cost (due to trashing penalty) will make this less attractive.
            # --- MODIFIED: Division by 1.0 (survival_rate_for_new_order) changes nothing ---
            cost_per_surviving_planted_unit = effective_cost_per_plant / survival_rate_for_new_order
            # --------------------------------------------------------------------------------

            orders_to_consider.append({
                'plant_type': pt,
                'qty_to_order': potential_order_qty_for_analysis, # This is the max quantity to consider for this type
                'nursery': best_nursery,
                'base_cost_per_unit': best_base_cost,
                'effective_cost_per_plant': effective_cost_per_plant,
                'cost_per_surviving_planted_unit': cost_per_surviving_planted_unit,
                'survival_rate': survival_rate_for_new_order, # This will be 1.0 for decision-making
                'expected_trashed_from_this_order': expected_trashed_from_this_order 
            })
        
        # 5. Sort potential orders by the calculated 'cost_per_surviving_planted_unit' (ascending)
        # This prioritizes the most cost-effective and least risky orders first.
        orders_to_consider.sort(key=lambda x: x['cost_per_surviving_planted_unit'])

        # 6. Place orders, strictly respecting all remaining capacities
        total_ordered_this_day = 0

        for order_info in orders_to_consider:
            # Stop if no storage space or no overall capacity for new orders remains
            if available_storage_space <= 0 or total_remaining_capacity_for_new_orders <= 0:
                break
            
            pt = order_info['plant_type']
            nursery = order_info['nursery']
            base_cost = order_info['base_cost_per_unit']

            # The actual quantity to order is the minimum of:
            # 1. The calculated 'qty_to_order' for this specific plant type (deficit capped by truck/storage)
            # 2. The *remaining* overall capacity for new orders across all types (prevents global over-ordering)
            # 3. The nursery truck capacity (explicitly added for clarity, though part of qty_to_order)
            
            order_qty = min(
                order_info['qty_to_order'],
                total_remaining_capacity_for_new_orders,
                self.NURSERY_TRUCK_CAPACITY 
            )
            
            if order_qty <= 0:
                continue # Skip if no quantity can be ordered

            # Place the order
            self.orders.append({
                'day': day,
                'nursery': nursery,
                'plant_type': pt,
                'quantity': order_qty,
                'arrival_day': day + self.ORDER_LEAD_TIME
            })
            
            # Add to in-transit inventory, storing original nursery cost for trashing calculation
            self.in_transit[pt].append((day + self.ORDER_LEAD_TIME, pt, order_qty, base_cost))
            # Add purchase cost and truck cost to total cost
            self.total_cost += order_qty * base_cost + self.NURSERY_TRUCK_COST 
            
            # Update available capacities
            available_storage_space -= order_qty
            total_remaining_capacity_for_new_orders -= order_qty # Consume from the global capacity pool
            total_ordered_this_day += order_qty

            # print(f"Day {day}: Ordered {order_qty} {pt} from {nursery} for arrival Day {day + self.ORDER_LEAD_TIME} (Base Cost: ${best_base_cost:.2f}/plant). Remaining cap for new orders: {total_remaining_capacity_for_new_orders}")

    def process_arrivals(self, day):
        """
        Processes plant arrivals from in-transit to storage. 
        Handles storage capacity limits by delaying excess arrivals.
        """
        # First, remove expired plants to free up space for new arrivals
        self.remove_expired_plants(day)

        for pt in list(self.in_transit.keys()): # Iterate over a copy of keys as list might change
            new_in_transit_list = [] # To hold plants that didn't arrive or were partially received
            arrived_today = 0
            
            # Sort in-transit batches by scheduled arrival day to process earlier ones first
            self.in_transit[pt].sort(key=lambda x: x[0]) 

            for arrival_day_scheduled, plant_type, qty, nursery_cost in self.in_transit[pt]:
                if arrival_day_scheduled == day: # This batch is scheduled to arrive today
                    current_storage_qty = sum(sum(q for q, _, _ in batches) for batches in self.storage.values())
                    space_left = self.STORAGE_CAPACITY - current_storage_qty
                    
                    if space_left > 0:
                        qty_to_store = min(space_left, qty)
                        # Add to storage, retaining arrival day and original nursery cost
                        self.storage[plant_type].append((qty_to_store, day, nursery_cost)) 
                        arrived_today += qty_to_store
                        remaining_qty = qty - qty_to_store
                        if remaining_qty > 0:
                            # If not all could be stored, reschedule remaining for tomorrow
                            new_in_transit_list.append((day + 1, plant_type, remaining_qty, nursery_cost)) 
                            # print(f"Day {day}: Accepted {qty_to_store} of {qty} {plant_type} plants. {remaining_qty} will attempt to arrive tomorrow due to storage limits.")
                    else:
                        # If no space, reschedule entire batch for tomorrow
                        new_in_transit_list.append((day + 1, plant_type, qty, nursery_cost)) 
                        # print(f"Day {day}: Storage full, couldn't accept {qty} {plant_type} plants. Will attempt to arrive tomorrow.")
                else:
                    # If not scheduled for today, keep it in in-transit list
                    new_in_transit_list.append((arrival_day_scheduled, plant_type, qty, nursery_cost))
            
            self.in_transit[pt] = new_in_transit_list
            
            if arrived_today > 0:
                print(f"Day {day}: {arrived_today} plants arrived and stored.")
        
    def remove_expired_plants(self, day):
        """
        Removes plants from storage that have exceeded their 7-day planting window.
        Applies the trashing penalty to the total cost.
        """
        removed_count = 0
        for pt in list(self.storage.keys()): # Iterate over a copy of keys as list might change
            updated_batches = [] # To hold batches that are still valid
            for qty, arrival_day, original_nursery_cost in self.storage[pt]:
                last_planting_possible_day = arrival_day + (self.PLANTING_END_DAY - 1)
                
                if day > last_planting_possible_day: # If current day is beyond the last possible planting day
                    removed_count += qty
                    # Add the additional penalty cost for trashing
                    self.total_cost += qty * self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST 
                    self.trashed_plants_count += qty 
                    print(f"Day {day}: Trashed {qty} {pt} plants that arrived on day {arrival_day} (expired). Original cost: ${original_nursery_cost:.2f}/plant + additional trashing penalty: ${self.PENALTY_TRASHED_PLANT_ADDITIONAL_COST:.2f}/plant.")
                else:
                    updated_batches.append((qty, arrival_day, original_nursery_cost))
            self.storage[pt] = updated_batches
            if not updated_batches:
                del self.storage[pt] # Remove plant type from storage if no batches remain

        if removed_count > 0:
            print(f"Day {day}: Total {removed_count} plants removed due to expiration.")

    def get_survival_rate(self, plant_type, current_day, arrival_day):
        """
        Get the survival rate for a plant_type based on the day of planting
        relative to its arrival day.
        
        MODIFIED: Returns 1.0 (100%) if within planting window, otherwise 0.0.
        The actual survival_rates data is ignored for decision-making but will be used for final reporting.
        """
        days_since_arrival = current_day - arrival_day + 1 # Calculate 1-indexed days since arrival
        
        # Check if planting is within the valid window
        if not (self.PLANTING_START_DAY <= days_since_arrival <= self.PLANTING_END_DAY):
            return 0.0 # Plant cannot be planted, effectively 0% survival
        
        # --- MODIFIED: Always return 1.0 if within planting window ---
        # The original line below would return the actual survival rate from the data:
        # planting_day_key = max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, days_since_arrival))
        # if plant_type in self.survival_rates.index and planting_day_key in self.survival_rates.columns:
        #    return self.survival_rates.loc[plant_type, planting_day_key]
        return 1.0 # Assume 100% survival for decision-making purposes when plantable
        # -------------------------------------------------------------


    def treat_and_plant_plants(self, day):
        """
        Manages the treatment and planting operations for a given day.
        Prioritizes plants close to expiration. Survival rate is ignored for prioritization.
        """
        day_type, max_hours = self.get_day_type(day)
        if day_type == 'sunday':
            # print(f"Day {day}: No planting on Sundays.")
            return # No operations on Sundays
            
        max_planting_minutes = max_hours * 60
        planting_minutes_used = 0
        
        eligible_plant_pool = [] # List of (priority_score, plant_type, qty, arrival_day_of_batch, original_nursery_cost)
        
        # Populate the pool with plants currently in storage that are eligible for planting
        for pt, batches in self.storage.items():
            for qty, arrival_day, original_nursery_cost in batches: 
                days_since_arrival = day - arrival_day + 1
                
                # Check if the plant is within its valid planting window
                if self.PLANTING_START_DAY <= days_since_arrival <= self.PLANTING_END_DAY:
                    # Retrieve the *actual* survival rate for reporting later, but not for decision making here
                    survival_day_key = max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, days_since_arrival))
                    if pt in self.survival_rates.index and survival_day_key in self.survival_rates.columns:
                        actual_current_day_survival_rate = self.survival_rates.loc[pt, survival_day_key]
                    else:
                        actual_current_day_survival_rate = 0.0

                    if actual_current_day_survival_rate > 0: # Still only consider if it has a chance to survive in reality
                        days_left_to_plant = (arrival_day + self.PLANTING_END_DAY - 1) - day
                        
                        # Priority score: lower 'days_left_to_plant' (more urgent)
                        # --- MODIFIED: Removed survival rate from priority score ---
                        priority_score = (days_left_to_plant, ) 
                        eligible_plant_pool.append((priority_score, pt, qty, arrival_day, original_nursery_cost, actual_current_day_survival_rate))
        
        # Sorts the pool based on the priority_score (ascending for days_left_to_plant)
        # Note: If two plants have the same days_left_to_plant, their relative order is stable (FIFO-ish based on input order)
        eligible_plant_pool.sort() 
        
        total_plants_in_eligible_pool = sum(item[2] for item in eligible_plant_pool) # Total quantity of eligible plants
        total_to_treat_today = min(
            total_plants_in_eligible_pool,
            self.TREATMENT_CAPACITY # Limited by daily treatment capacity
        )
        
        if total_to_treat_today <= 0:
            # print(f"Day {day}: No eligible plants available in storage to treat.")
            return # No plants to treat/plant today
        
        treated_today = 0 # Counter for plants treated on the current day
        
        # Iterate through the prioritized eligible plants
        for priority_score, pt, qty_in_batch, arrival_day_of_batch, original_nursery_cost, actual_survival_rate_for_record in eligible_plant_pool:
            # Stop if daily treatment capacity or working hours are exhausted
            if treated_today >= total_to_treat_today or planting_minutes_used >= max_planting_minutes:
                break 
            
            polygons_with_need = [] # List of (polygon, remaining_need)
            # Find polygons that need this plant type and where it can be planted
            for p in self.polygons:
                remaining_need = max(0, self.plant_requirements[p].get(pt, 0) - self.planted_per_polygon[p].get(pt, 0))
                if remaining_need > 0:
                    # We still check if the *actual* survival rate for this polygon/plant type is > 0 for this day.
                    # This prevents trying to plant where it's truly impossible to survive.
                    polygon_actual_survival_rate = self.survival_rates.loc[pt, max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, day - arrival_day_of_batch + 1))] \
                        if pt in self.survival_rates.index and max(self.PLANTING_START_DAY, min(self.PLANTING_END_DAY, day - arrival_day_of_batch + 1)) in self.survival_rates.columns else 0.0

                    if polygon_actual_survival_rate > 0: 
                        polygons_with_need.append((p, remaining_need, polygon_actual_survival_rate)) # Pass actual survival for recording
            
            # Sort polygons by quantity needed (more first), as survival rate is not a factor for decision-making
            # We explicitly *do not* sort by the actual_survival_rate here for decision purposes.
            polygons_with_need.sort(key=lambda x: x[1], reverse=True) 

            plants_from_this_batch_planted = 0 # Counter for plants planted from the current batch
            
            for p, remaining_need_in_polygon, polygon_actual_survival_rate_for_record in polygons_with_need:
                # Stop if current batch is fully planted, or daily capacity/hours exhausted
                if plants_from_this_batch_planted >= qty_in_batch or treated_today >= total_to_treat_today or planting_minutes_used >= max_planting_minutes:
                    break
                
                # Calculate delivery time to the polygon
                delivery_time = 60 + 2 * self.distance_matrix.loc[self.storage_polygon, p]
                # Check if there's enough time left for this delivery
                if planting_minutes_used + delivery_time > max_planting_minutes:
                    continue # Skip if not enough time
                
                # Determine the quantity to plant in this polygon from this batch
                plant_qty = min(
                    remaining_need_in_polygon,        # Cannot plant more than polygon needs
                    qty_in_batch - plants_from_this_batch_planted, # Cannot plant more than available in this batch
                    total_to_treat_today - treated_today,          # Cannot exceed overall daily treatment capacity
                    self.DELIVERY_TRUCK_CAPACITY         # Cannot exceed delivery truck capacity
                )
                
                if plant_qty <= 0:
                    continue # Skip if no quantity to plant

                # Update storage: find the exact batch and reduce its quantity
                batch_found = False
                for i, (stored_qty, stored_arrival_day, stored_nursery_cost) in enumerate(self.storage[pt]):
                    # Match by arrival_day and original_nursery_cost to ensure it's the correct batch
                    if stored_arrival_day == arrival_day_of_batch and stored_nursery_cost == original_nursery_cost: 
                        self.storage[pt][i] = (stored_qty - plant_qty, stored_arrival_day, stored_nursery_cost)
                        if self.storage[pt][i][0] <= 0: # If batch quantity drops to zero or less, remove it
                            self.storage[pt].pop(i)
                        batch_found = True
                        break
                if not batch_found:
                    print(f"Warning: Batch not found for {pt} with arrival_day {arrival_day_of_batch} and cost {original_nursery_cost}. This should not happen if logic is sound.")

                plants_from_this_batch_planted += plant_qty
                treated_today += plant_qty
                
                # Record the planting event, using the actual survival rate for the record
                self.plantings.append({
                    'day': day,
                    'polygon': p,
                    'plant_type': pt,
                    'quantity': plant_qty,
                    'arrival_day_of_batch': arrival_day_of_batch,
                    'survival_rate_applied': polygon_actual_survival_rate_for_record 
                })
                
                self.total_planted += plant_qty
                self.planted_per_polygon[p][pt] += plant_qty
                # Add planting cost and delivery truck cost to total cost
                self.total_cost += plant_qty * self.PLANTING_COST_PER_PLANT + self.DELIVERY_TRUCK_COST 
                
                planting_minutes_used += delivery_time

                # Print the trip time for the current delivery
                print(f"Day {day}: Trip to {p} for {plant_qty} plants took {delivery_time} minutes.")

                # Update for average survival rate calculation using the ACTUAL survival rate
                self.total_expected_survivors += plant_qty * polygon_actual_survival_rate_for_record
                self.total_quantity_planted_for_avg_survival += plant_qty
                
                # print(f"Day {day}: Planted {plant_qty} {pt} at {p} (from batch arr. day {arrival_day_of_batch}, survival: {polygon_actual_survival_rate_for_record:.2f}, delivery: {delivery_time} mins)")
        
        print(f"Day {day}: Total treated and planted today: {treated_today}/{total_to_treat_today}.")


    def run_heuristic(self):
        """Runs the complete heuristic simulation for the specified number of days."""
        print("Running heuristic solution...")
        start_time = time.time()
        
        self.load_data()
        self.initialize_structures()
        
        for day in range(self.DAYS):
            print(f"\n=== Day {day} ===")
            
            self.process_arrivals(day) # Process plants arriving today
            self.order_plants(day) # Place new orders
            self.treat_and_plant_plants(day) # Treat and plant plants from storage
            
            # Print daily summary
            current_storage_qty = sum(sum(qty for qty, _, _ in batches) for batches in self.storage.values())
            print(f"Current Storage: {current_storage_qty}/{self.STORAGE_CAPACITY}")
            print(f"Total planted so far: {self.total_planted}")
            print(f"Total trashed plants: {self.trashed_plants_count}")
            
            # Check if all requirements are met to potentially stop early
            all_met = True
            for p in self.polygons:
                for pt in self.plant_types:
                    req = self.plant_requirements[p].get(pt, 0)
                    planted = self.planted_per_polygon[p].get(pt, 0)
                    if planted < req:
                        all_met = False
                        break
                if not all_met:
                    break
            
            if all_met and current_storage_qty == 0 and sum(sum(qty for _, _, qty, _ in transit_items) for transit_items in self.in_transit.values()) == 0:
                print("\nAll plants have been planted and no pending deliveries! Stopping simulation early.")
                break
            elif all_met:
                print("\nAll required plants have been planted, but there are still plants in storage or transit. Continuing to clear existing stock.")
                # The order_plants function will naturally stop ordering if requirements are met,
                # so the simulation will continue to clear existing inventory.

        # Calculate average survival rate at the end of the simulation
        average_survival_rate = 0.0
        if self.total_quantity_planted_for_avg_survival > 0:
            average_survival_rate = self.total_expected_survivors / self.total_quantity_planted_for_avg_survival

        total_required = sum(sum(p.values()) for p in self.plant_requirements.values())
        completion = (self.total_planted / total_required) * 100 if total_required > 0 else 0
        
        print("\n=== Heuristic Solution Complete ===")
        print(f"Total Cost: ${self.total_cost:,.2f}")
        print(f"Plants Planted: {self.total_planted:,} of {total_required:,}")
        print(f"Completion: {completion:.1f}%")
        print(f"Total Trashed Plants: {self.trashed_plants_count:,}") 
        print(f"Average Survival Rate of Planted Plants: {average_survival_rate:.2%}") # Formatted as percentage
        print(f"Runtime: {time.time() - start_time:.2f} seconds")
        
        self.save_results(average_survival_rate) # Pass average survival rate to save_results

    def save_results(self, average_survival_rate):
        """Save simulation results to CSV files."""
        print("\nSaving results...")
        
        orders_df = pd.DataFrame(self.orders)
        if not orders_df.empty:
            orders_df.to_csv('heuristic_orders.csv', index=False)
            print(f"Saved {len(orders_df)} orders to heuristic_orders.csv.")
        else:
            print("No orders were placed, 'heuristic_orders.csv' not created.")
        
        planting_df = pd.DataFrame(self.plantings)
        if not planting_df.empty:
            planting_df.to_csv('heuristic_planting.csv', index=False)
            print(f"Saved {len(planting_df)} planting records to heuristic_planting.csv.")
        else:
            print("No plants were planted, 'heuristic_planting.csv' not created.")
        
        total_required = sum(sum(p.values()) for p in self.plant_requirements.values())
        summary = {
            'total_cost': self.total_cost,
            'total_planted': self.total_planted,
            'total_required': total_required,
            'completion_percentage': (self.total_planted / total_required) * 100 if total_required > 0 else 0,
            'total_trashed_plants': self.trashed_plants_count,
            'average_survival_rate_planted': average_survival_rate # Add to summary
        }
        pd.DataFrame([summary]).to_csv('heuristic_summary.csv', index=False)
        print("Saved summary to heuristic_summary.csv.")

if __name__ == "__main__":
    solver = ReforestationHeuristic()
    solver.run_heuristic()